In [6]:
%load_ext autoreload
%autoreload 2
from git_analysis import CommitProcessor
import pygit2
import itertools
from pygit2 import GIT_SORT_TOPOLOGICAL, GIT_SORT_REVERSE
repo = pygit2.Repository("../jadx")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
ext = CommitProcessor("../jadx")
ext.add_commit("a27ba3ff4be70a1d46e9065151ef36850686dc39")
ext.commits[0]

visiting commit <pygit2.Object{commit:a27ba3ff4be70a1d46e9065151ef36850686dc39}> by Skylot 


[autoreload of git_analysis.commit_processor failed: Traceback (most recent call last):
  File "c:\dev\gitAnalysis\venv\lib\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "c:\dev\gitAnalysis\venv\lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "C:\Users\Danny\scoop\apps\python\current\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\dev\gitAnalysis\git_analysis\commit_processor.py", line 2, in <module>
    from java_change_detector import JavaChangeDetector, JavaHierarchy
ModuleNotFoundError: No module named 'java_change_detector'
]


{'author': 'skylot@gmail.com',
 'committer': 'skylot@gmail.com',
 'message': "fix(res): skip '.9.png' decode if patch data not found (#1112)\n",
 'id': a27ba3ff4be70a1d46e9065151ef36850686dc39,
 'patches': [{'from': 'jadx-core/src/main/java/jadx/api/ResourcesLoader.java',
   'to': 'jadx-core/src/main/java/jadx/api/ResourcesLoader.java',
   'status': 'M',
   'hunks': [{'lines': [{'content': '\t\tif (name.endsWith(".9.png")) {\n'},
      {'content': '\t\t\ttry (ByteArrayOutputStream os = new ByteArrayOutputStream()) {\n'},
      {'content': '\t\t\t\tRes9patchStreamDecoder decoder = new Res9patchStreamDecoder();\n'},
      {'content': '\t\t\t\tif (decoder.decode(inputStream, os)) {\n'},
      {'content': '\t\t\t\t\treturn ResContainer.decodedData(rf.getDeobfName(), os.toByteArray());\n'},
      {'content': '\t\t\t\t}\n'},
      {'content': '\t\t\t\tdecoder.decode(inputStream, os);\n'},
      {'content': '\t\t\t\treturn ResContainer.decodedData(rf.getDeobfName(), os.toByteArray());\n'},
  

In [129]:
walk = repo.walk(repo.head.target, GIT_SORT_TOPOLOGICAL)
for commit in itertools.islice(walk, 1):
    for parent in commit.parents:
        diff = repo.diff(commit.tree_id, parent.tree_id, context_lines=3)
        print(f"dif between {commit.id} and {parent.id}")
        for patch in diff:
            print(patch.data)

dif between a72523c7dff91a9bfe6066e72f18cc540bff9099 and 46eeb0bc22a71d96f8fc18d48061f28fe2766c30
b"diff --git a/README.md b/README.md\nindex c1a164d..030bae1 100644\n--- a/README.md\n+++ b/README.md\n@@ -11,8 +11,6 @@\n \n Command line and GUI tools for producing Java source code from Android Dex and Apk files\n \n-:exclamation: :exclamation: :exclamation: Please note that in most cases Jadx can't decompile all 100% of the code, so errors will occur. Check [Troubleshooting guide](https://github.com/skylot/jadx/wiki/Troubleshooting-Q&A#decompilation-issues) for workarounds\n-\n **Main features:**\n - decompile Dalvik bytecode to java classes from APK, dex, aar, aab and zip files\n - decode `AndroidManifest.xml` and other resources from `resources.arsc`\n"


In [133]:
walk = repo.walk(repo.head.target, GIT_SORT_TOPOLOGICAL)
commit = next(walk)
parent = commit.parents[0]
commit, parent
diff = repo.diff(commit, parent)
patches = list(diff)
deltas = list(diff.deltas)
patch = patches[0]
delta = deltas[0]
hunk = patch.hunks[0]

In [135]:

commits = {}

def visit_commit(commit):
    print(f"visiting commit {commit} by {commit.author.name} ")
    visit = {
        "author": commit.author.email,
        "committer": commit.committer.email,
        "message": commit.message,
        "id": commit.id
    }
    for parent in commit.parents:
        diff = repo.diff(commit, parent)
        visit["patches"] = []
        for patch in diff:
            delta = patch.delta
            patch_object = {
                "from": delta.old_file.path, "to": delta.new_file.path,
                "status": delta.status_char(),
                "hunks": []
            }
            for hunk in patch.hunks:
                hunk_object = { "lines": []}
                for line in hunk.lines:
                    hunk_object["lines"].append({
                        "content": line.content
                    })
                patch_object["hunks"].append(hunk_object)
            visit["patches"].append(patch_object)

    commits[visit["id"]] = visit


visit_commit(commit)
commits

visiting commit <pygit2.Object{commit:a72523c7dff91a9bfe6066e72f18cc540bff9099}> by skylot 


{a72523c7dff91a9bfe6066e72f18cc540bff9099: {'author': '118523+skylot@users.noreply.github.com',
  'committer': 'skylot@gmail.com',
  'message': 'docs: add link to decompilation troubleshooting\n',
  'id': a72523c7dff91a9bfe6066e72f18cc540bff9099,
  'patches': [{'from': 'README.md',
    'to': 'README.md',
    'status': 'M',
    'hunks': [{'lines': [{'content': '\n'},
       {'content': 'Command line and GUI tools for producing Java source code from Android Dex and Apk files\n'},
       {'content': '\n'},
       {'content': ":exclamation: :exclamation: :exclamation: Please note that in most cases Jadx can't decompile all 100% of the code, so errors will occur. Check [Troubleshooting guide](https://github.com/skylot/jadx/wiki/Troubleshooting-Q&A#decompilation-issues) for workarounds\n"},
       {'content': '\n'},
       {'content': '**Main features:**\n'},
       {'content': '- decompile Dalvik bytecode to java classes from APK, dex, aar, aab and zip files\n'},
       {'content': '- decod